In [1]:
import pandas as pd
import sklearn

In [2]:
##This parts reads in the files and joins them

In [3]:
engagements_tc = pd.read_csv('engagements_tc.txt',delimiter="\t")

In [4]:
print engagements_tc.columns

Index([u'post_id', u'events', u'engagement_date'], dtype='object')


In [5]:
meta_tc = pd.read_csv('meta_tc.txt',delimiter='\t+')

/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


meta_tc.columns

In [6]:
meta_tc.set_index('post_id',inplace=True)
engagements_tc.set_index('post_id',inplace=True)

In [7]:
engagements_meta = pd.merge(meta_tc,engagements_tc,how='inner',left_index=True,right_index=True)

In [8]:
engagements_meta.columns

Index([u'link', u'title', u'summary', u'author_actor_id', u'pub_date',
       u'events', u'engagement_date'],
      dtype='object')

In [9]:
engagements_meta['engagement_date'] = engagements_meta['engagement_date'].astype(float)

In [10]:
engagements_meta.dtypes

link                object
title               object
summary             object
author_actor_id    float64
pub_date            object
events               int64
engagement_date    float64
dtype: object

In [11]:
import time
from datetime import datetime

In [12]:
def convertEngagementDate(x):
    x[6] = datetime.fromtimestamp(x[6]).strftime('%Y-%m-%d %H:%M:%S')
    #x[6] = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x[6]))
    return x

In [13]:
engagements_meta = engagements_meta.apply(convertEngagementDate,axis=1)

In [14]:
def convertPubDate(x):
    x[4] = datetime.strptime(x[4], "%Y-%m-%d %H:%M:%S")
    return x

In [15]:
engagements_meta.dtypes

link                object
title               object
summary             object
author_actor_id    float64
pub_date            object
events               int64
engagement_date     object
dtype: object

In [16]:
engagements_meta = engagements_meta.apply(convertPubDate,axis=1)

In [17]:
def dateDifference(x): 
    x[6] = datetime.strptime(str(x[6]), "%Y-%m-%d %H:%M:%S")
    x[7] = abs((x[6] - x[4]).days)
    return x
engagements_meta.dtypes

link                       object
title                      object
summary                    object
author_actor_id           float64
pub_date           datetime64[ns]
events                      int64
engagement_date            object
dtype: object

In [18]:
# PART 2
# Feature Extraction
# Since we have to find views for the first day we find the events for the first day
# Do analysis on link url - Analysis Results : All the events came from one view
# Find AuthorEventsRation = Views by Author/ Total number of views
# LDA to find topics for the authors - Could not be completed because of issues with one hot encoding. If resolved could have
# been used as feature

In [19]:
engagements_meta['date_difference'] = 1
engagements_meta = engagements_meta.apply(dateDifference,axis=1)

In [20]:
authorEventCount = engagements_meta.groupby('author_actor_id')['events'].sum()

In [21]:
authorEventCount = pd.DataFrame(authorEventCount)
authorEventCount.columns = ['Total_events']
authorEventCount.head(1)
totalEventCount = authorEventCount['Total_events'].sum()
authorEventCount.columns = ['AuthorEventsRatio']

In [22]:
def calculateEventCountRatio(x):
    x = (x*1.0)/(totalEventCount*1.0)
    return x

In [23]:
authorEventCount = authorEventCount.applymap(lambda x : calculateEventCountRatio(x))

In [24]:
authorEventCount.head(1)
authorEventCount.to_csv('AuthorEventsRatio.csv')

In [25]:
engagements_meta['post_id'] = engagements_meta.index
engagements_meta.set_index('author_actor_id',inplace=True)
engagements_meta_author_ratio = pd.merge(authorEventCount,engagements_meta,left_index=True,right_index=True, how="inner")

In [26]:
engagements_meta_author_ratio.columns

Index([u'AuthorEventsRatio', u'link', u'title', u'summary', u'pub_date',
       u'events', u'engagement_date', u'date_difference', u'post_id'],
      dtype='object')

In [27]:
engagements_meta_author_ratio = engagements_meta_author_ratio.sort(['AuthorEventsRatio','events'],ascending=[0,0])

/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [28]:
#
# Finding similarity between content of same users
#

In [35]:
similarityDf = pd.merge(engagements_meta_author_ratio,engagements_meta_author_ratio,left_index=True,right_index=True,how='inner')

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib
import numpy as np
vectorizer = TfidfVectorizer(min_df=1,stop_words='english',ngram_range=(2, 2))
tfidf_1 = vectorizer.fit_transform(engagements_meta_author_ratio['summary'].values.tolist())
joblib.dump(tfidf_1,"vectorizer.pk")

/Library/Python/2.7/site-packages/numpy-1.11.0-py2.7-macosx-10.10-intel.egg/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


['vectorizer.pk',
 'vectorizer.pk_01.npy',
 'vectorizer.pk_02.npy',
 'vectorizer.pk_03.npy']

In [38]:
similarityDf = tfidf_1 * tfidf_1.T
similarityArray = similarityDf.todense()
postId = engagements_meta_author_ratio['post_id'].values.tolist()

In [48]:
np.hstack((similarityArray,postId))

ValueError: all the input arrays must have same number of dimensions

30209

In [ ]:
engagements_meta_author_ratio.to_csv('engagements_meta_author_ratio.csv')

In [ ]:
#
# Link Analysis - Did not give sufficient value - Events from only one url
#

In [ ]:
linkGrouping = engagements_meta_author_ratio['link']
linkGrouping = pd.DataFrame(linkGrouping)
linkGrouping.columns

In [ ]:
from urlparse import urlparse
def extractLinkUrl(x):
    parsed_uri = urlparse(x)
    domain = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    return domain

In [ ]:
linkGrouping = linkGrouping.applymap(lambda x: extractLinkUrl(x))
linkGrouping.drop_duplicates(['link'])

In [ ]:
#
# START LDA WORK
# Could not be completed because of problems with one hot encoding
#

In [ ]:
summary = engagements_meta_author_ratio[['summary','post_id']]

summary.head(1)

In [ ]:
summary = pd.DataFrame(summary)

In [ ]:
summary.head(5)

In [ ]:
summary['author_actor_id'] = summary.index

In [ ]:
summaryList = summary.values.tolist()

summaryList[1]

In [ ]:
print summaryList[4]

In [ ]:
AuthorTextIndex = []
def combineText():
    startIndex = int(summaryList[1][1])
    text = ''
    for x in summaryList:
        if int(x[1]) == startIndex:
            text = text + x[0]
        elif int(x[1]) != startIndex:
            AuthorTextIndex.append((startIndex,text))
            startIndex = int(x[1])
            text = x[0]

combineText()          
            

In [ ]:
len(AuthorTextIndex)

In [ ]:
AuthorTextIndex[2]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
vectorizer = TfidfVectorizer(min_df=1,stop_words='english',ngram_range=(2, 2))

In [ ]:
AuthorText = [x[1] for x in  AuthorTextIndex]

In [ ]:
print len(AuthorText)

In [ ]:
AuthorTextVectorizer = vectorizer.fit_transform(AuthorText)
idf = vectorizer.idf_
idf.sort

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation


In [ ]:
#nmf = NMF(n_components=10, random_state=1, alpha=.1, l1_ratio=.5).fit(AuthorTextVectorizer)
topicList = []
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        s = " ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]])
        topicList.append(s)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [ ]:
lda = LatentDirichletAllocation(n_topics=1000, max_iter=5,
                                learning_method='online', learning_offset=50.,
                                random_state=0)

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(AuthorText)

In [ ]:
lda.fit(tf)

In [ ]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names,5)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
label_encoder_list = " ".join(topicList)
label_encoder_list = label_encoder_list.split(' ')
len(label_encoder_list)

In [ ]:
le.fit(label_encoder_list)
label_encoder_set = set(label_encoder_list)
label_encoder_df = pd.DataFrame(list(label_encoder_set))
label_encoder_df.to_csv('label_encoder.csv')

In [ ]:
AuthorTextCopy = [unicode(x[0],"utf-8") for x in summaryList]
summaryReducedTopics = []
for x in AuthorTextCopy:
    tempList = x.split(" ")
    topicReduced = " ".join([str(val) for val in tempList if val in label_encoder_list])
    if topicReduced != '':
        summaryReducedTopics.append(topicReduced)

In [ ]:
from sklearn.externals import joblib
joblib.dump(le, 'lda.pkl')

In [ ]:
le.transform(summaryReducedTopics)

In [ ]:
engagements_meta_author_ratio

In [ ]:
datasetDf = engagements_meta_author_ratio[(engagements_meta_author_ratio['date_difference'] <= 1)]

In [ ]:
datasetDf

In [ ]:
#
# BUILDING THE REGRESSION MODEL
# Using SVR with Rbf as final model
#

In [ ]:
datasetDf = datasetDf[['AuthorEventsRatio','summary','date_difference','events','post_id']]

In [ ]:
from sklearn.preprocessing import OneHotEncoder
#enc.categorical_features()
ohefeatures = []
for x in AuthorTextCopy:
    tempList = x.split(" ")
    topicReduced = [val for val in tempList if val in label_encoder_list]
    if topicReduced != '':
        ohefeatures.append(topicReduced)

In [ ]:
ohefeaturesFlattened = sum(ohefeatures,[])
#enc.fit(ohefeatures)
#enc_topics = enc.transform(summaryReducedTopics)

In [ ]:
enc = preprocessing.LabelEncoder()
list(set(ohefeaturesFlattened))
ohefeaturesArray = list(set(ohefeaturesFlattened))

In [ ]:
ohefeaturesArray
#enc.fit(ohefeaturesArray)
enc.fit(ohefeaturesArray)
enc.transform(ohefeatures)
#ohefeatures

In [ ]:
datasetDf['author_actor_id'] = datasetDf.index
X = datasetDf[['author_actor_id','date_difference','AuthorEventsRatio']]
X = X.values.tolist()
X = np.array(X)
print similarityDf.shape
print X.shape
X = np.hstack((X,similarityDf))

In [ ]:
from sklearn.cross_validation import train_test_split
import numpy as np
y = datasetDf['events']
y = y.values.tolist()
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.grid_search import GridSearchCV
svr = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=5,
                   param_grid={"C": [1e0, 1e1, 1e2, 1e3],
                               "gamma": np.logspace(-2, 2, 5)})
kr = GridSearchCV(KernelRidge(kernel='rbf', gamma=0.1), cv=5,
                  param_grid={"alpha": [1e0, 0.1, 1e-2, 1e-3],
                              "gamma": np.logspace(-2, 2, 5)})
svr.fit(X_train, y_train)
kr.fit(X_train, y_train)

In [ ]:
y_pred_svr = svr.predict(X_test)
y_pred_kr = kr.predict(X_test)
from sklearn import metrics
print metrics.mean_squared_error(y_test,y_pred_svr)
print metrics.mean_squared_error(y_test,y_pred_kr)

In [ ]:
#Since svr with rbf gives better performance we save the svr model
joblib.dump(svr,"svr.pk")